<a href="https://colab.research.google.com/github/obedotto/rl-value-iteration/blob/main/RL_Value_Iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
np.bool = bool  # Fix numpy compatibility issue


In [4]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-m_dt9ub3/gym-walk_0c804b8474924bebb524fdd7225414cc
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-m_dt9ub3/gym-walk_0c804b8474924bebb524fdd7225414cc
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=5377 sha256=6638e4b4aa6220c7726989b078f990847dfa75f21d26a5541ad496cc57762c23
  Stored in directory: /tmp/pip-ephem-wheel-cache-2k6dry3c/wheels/bf/23/e5/a94be4a90dd18f7ce958c21f192276cb01ef0daaf2bc66583b
Successfully built gym-walk


In [5]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [6]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [7]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123)
    np.random.seed(123)
    env.reset(seed=123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, _ = env.step(pi[state])  # FIXED: pi(state) → pi[state]
            steps += 1
        results.append(state == goal_state)
    return np.sum(results) / len(results)

In [8]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123)
    np.random.seed(123)
    env.reset(seed=123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi[state])  # FIXED: pi(state) → pi[state]
            results[-1] += reward
            steps += 1
    return np.mean(results)

# Creating the Frozen Lake environment

In [9]:
envdesc  = ['SFHH','HFFH','HHFH', 'HGFH']
env = gym.make('FrozenLake-v1',desc=envdesc)
init_state = env.reset()
goal_state = 13
P = env.env.P

In [10]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, True),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, True),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, False),
   (0.3333333333333333, 2, 0.0, True)],
  2: [(0.3333333333333333, 5, 0.0, False),
   (0.3333333333333333, 2, 0.0, True),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, True),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2: {0

# Value Iteration Algorithm

In [11]:
def value_iteration(P, gamma=1.0, theta=1e-10):
    V = np.zeros(len(P), dtype=np.float64)   # Initialize state-value function
    pi = np.zeros(len(P), dtype=int)         # Initialize policy (best action per state)

    while True:
        delta = 0
        for s in range(len(P)):  # Loop over all states
            v = V[s]
            q_values = []
            for a in P[s]:  # Loop over possible actions
                q_sa = 0
                for prob, next_s, reward, done in P[s][a]:
                    q_sa += prob * (reward + gamma * V[next_s])
                q_values.append(q_sa)

            V[s] = max(q_values)              # Update state-value
            pi[s] = np.argmax(q_values)       # Update optimal action
            delta = max(delta, abs(v - V[s]))  # Track convergence

        if delta < theta:  # Stop when values converge sufficiently
            break

    return V, pi


In [12]:
# Finding the optimal policy
V_best_v, pi_best_v = value_iteration(P, gamma=0.99)


In [13]:
# Printing the policy
print("Name: MIDHUN AZHAHU RAJA P Register Number: 212222240066")
print('Optimal policy and state-value function (VI):')
print_policy(pi_best_v, P)

Name: MIDHUN AZHAHU RAJA P Register Number: 212222240066
Optimal policy and state-value function (VI):
Policy:
| 00      ^ | 01      < |           |           |
|           | 05      > | 06      < |           |
|           |           | 10      < |           |
|           |           | 14      < |           |


In [15]:
# printing the success rate and the mean return
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_best_v, goal_state=goal_state)*100,
    mean_return(env, pi_best_v)))

Reaches goal 5.00%. Obtains an average undiscounted return of 0.0500.


In [16]:
# printing the state value function
print_state_value_function(V_best_v, P, prec=4)

State-value function:
| 00 0.0399 | 01 0.0411 |           |           |
|           | 05 0.0436 | 06 0.0909 |           |
|           |           | 10 0.2319 |           |
|           |           | 14 0.6117 |           |
